In [25]:
#installing the packages
!pip install BeautifulSoup4
!pip install lxml
!pip install html5lib
!pip install geocoder

  Using cached https://files.pythonhosted.org/packages/21/0a/47fdf541c97fd9b6a610cb5fd518175308a7cc60569962e776ac52420387/beautifulsoup4-4.6.3-py3-none-any.whl
Target directory /home/spark/shared/user-libs/python3/beautifulsoup4-4.6.3.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/bs4 already exists. Specify --upgrade to force replacement.
  Using cached https://files.pythonhosted.org/packages/9e/3a/32438b5b54d2c13a9be88d2718b108eb6f3eb501fc68726b9faadebae160/lxml-4.2.4-cp35-cp35m-manylinux1_x86_64.whl
Target directory /home/spark/shared/user-libs/python3/lxml-4.2.4.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/lxml already exists. Specify --upgrade to force replacement.
  Using cached https://files.pythonhosted.org/packages/a5/62/bbd2be0e7943ec8504b517e62bab011b4946e1258842bc159e5dfde15b96/html5lib-1.0.1-py2.py3-none-any.whl
  Using cached 

In [27]:
#loading the liberaries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import geocoder

In [28]:
canadapostalcodes = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [29]:
#loading the request url
canadapostalcodeshtml = requests.get(canadapostalcodes).text
soup = BeautifulSoup(canadapostalcodeshtml,'lxml')

In [30]:
#loading the  table data
matched_data = soup.find_all('table', class_="wikitable sortable")

In [31]:
#defining the table columns
df = pd.DataFrame(columns=[th.text.replace('\n','') for th in matched_data[0].find_all('tr')[0].find_all('th')])

In [32]:
#loading the data for each column
df[df.columns[0]] = [tr.find_all('td')[0].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]
df[df.columns[1]] = [tr.find_all('td')[1].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]
df[df.columns[2]] = [tr.find_all('td')[2].text.replace('\n','') for tr in matched_data[0].find_all('tr') if len(tr.find_all('td'))>0]

In [33]:
#filtering the Not assigned data and null data
complete_info_df =df[(df[df.columns[0]] != 'Not assigned') & (df[df.columns[1]] != 'Not assigned') & (df[df.columns[2]] != 'Not assigned')]
complete_info_df=complete_info_df.dropna()
complete_info_df.reset_index(drop=True, inplace=True)
#pd.set_option('display.max_rows', 500)

In [35]:
#merging the repeated
complete_info_df=complete_info_df.groupby([complete_info_df.columns[0],complete_info_df.columns[1]], as_index=False)['Neighbourhood'].agg(','.join)

In [96]:
complete_info_df.Neighbourhood = complete_info_df.Neighbourhood.apply(lambda x: x.replace(']',''))
complete_info_df.shape
#completed assignment 1

(102, 3)

In [97]:
#geocoder.google('M1B , Scarborough , Toronto , Ontario, Canada').latlng

In [169]:
#getting the lattiude and longtidude
longlatlist = complete_info_df.apply(lambda x:geocoder.google('{}, {}, Toronto, Ontario'.format(x['Postcode'], x['Borough'])).latlng, axis=1)
#longlatlist

0                                         None
1             [43.78453510000001, -79.1604971]
2                                         None
3                    [43.7709921, -79.2169174]
4              [43.773136, -79.23947609999999]
5                                         None
6                                         None
7                    [43.7111117, -79.2845772]
8              [43.716316, -79.23947609999999]
9                     [43.692657, -79.2648481]
10                    [43.7574096, -79.273304]
11                   [43.7500715, -79.2958491]
12                   [43.7942003, -79.2620294]
13                                        None
14                                        None
15                                        None
16            [43.8361247, -79.20563609999999]
17                                        None
18                   [43.7785175, -79.3465557]
19                    [43.7869473, -79.385975]
20                                        None
21           

In [172]:
complete_info_df['LongLat'] = longlatlist

In [177]:
complete_info_df.dropna(inplace=True)

In [182]:
complete_info_df['Lontiude'] = complete_info_df.LongLat.apply(lambda x: x[1])

In [183]:
complete_info_df['Latitude'] = complete_info_df.LongLat.apply(lambda x: x[0])

In [187]:
complete_info_df.drop(columns='LongLat',inplace=True)

In [188]:
complete_info_df

,Postcode,Borough,Neighbourhood,Lontiude,Latitude
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",-79.160497,43.784535
3,M1G,Scarborough,Woburn,-79.216917,43.770992
4,M1H,Scarborough,Cedarbrae,-79.239476,43.773136
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",-79.284577,43.711112
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",-79.239476,43.716316
9,M1N,Scarborough,"Birch Cliff,Cliffside West",-79.264848,43.692657
10,M1P,Scarborough,"Dorset Park,Scarborough Town Centre,Wexford He...",-79.273304,43.757410
11,M1R,Scarborough,"Maryvale,Wexford",-79.295849,43.750071
12,M1S,Scarborough,Agincourt,-79.262029,43.794200
16,M1X,Scarborough,Upper Rouge,-79.205636,43.836125


In [189]:
#end of assignment 2